In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [4]:
# === 1. VERİYİ YÜKLE ===
df = pd.read_csv("../prediction_model/data/data.csv", encoding="ISO-8859-1", sep=";")

# === 2. Hedef Değişkeni Grupla ===
def map_ktas(value):
    if value in [1, 2]:
        return 0  # kırmızı
    elif value == 3:
        return 1  # sarı
    elif value in [4, 5]:
        return 2  # yeşil
    return np.nan

df['target'] = df['KTAS_expert'].apply(map_ktas)

# === 3. Veri Temizliği ===
df = df.replace(",", ".", regex=True)  # sayısal virgülleri noktaya çevir
df = df.drop(columns=['Chief_complain', 'Diagnosis in ED', 'KTAS_expert'])
df = df.apply(pd.to_numeric, errors='coerce')  # tüm veriyi sayısala çevir
df = df.fillna(df.mean(numeric_only=True))     # eksikleri ortalama ile doldur

In [5]:
def check_df(dataframe):
    """
    Checks the overall structure and key metrics of a DataFrame.

    Args:
        dataframe (pd.DataFrame): DataFrame to inspect.

    Returns:
        None: Prints shape, data types, head, tail, missing values, and quantiles.
    """
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print('##################### Unique Values #####################')
    print(dataframe.nunique())
    print("##################### Quantiles #####################")
    # Uncomment below to include quantile information
    print(dataframe[[col for col in dataframe.columns if dataframe[col].dtypes != "O"]].quantile([0, 0.05, 0.50, 0.75, 0.95, 0.99, 1]).T)
    #print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)


check_df(df)

##################### Shape #####################
(1267, 22)
##################### Types #####################
Group                         int64
Sex                           int64
Age                           int64
Patients number per hour      int64
Arrival mode                  int64
Injury                        int64
Mental                        int64
Pain                          int64
NRS_pain                    float64
SBP                         float64
DBP                         float64
HR                          float64
RR                          float64
BT                          float64
Saturation                  float64
KTAS_RN                       int64
Disposition                   int64
Error_group                   int64
Length of stay_min            int64
KTAS duration_min           float64
mistriage                     int64
target                        int64
dtype: object
##################### Head #####################
   Group  Sex  Age  Patients numbe

In [ ]:
# === 4. Özellik ve Etiket Ayır ===
X = df.drop(columns=['target'])
y = df['target']

# === 5. Eğitim/Test Ayrımı ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# === 6. Normalizasyon ===
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# === 7. One-hot encoding (Keras için) ===
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)

# === 8. MLP Modeli ===
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# === 9. Eğit ===
model.fit(X_train, y_train_cat, epochs=30, batch_size=32, validation_split=0.1)

# === 10. Değerlendirme ===
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print(classification_report(y_test, y_pred, target_names=["Kırmızı", "Sarı", "Yeşil"]))
# === 11. Modeli ve scaler'ı kaydet ===
from tensorflow.keras.models import load_model
import joblib

# Modeli kaydet (HDF5 formatı)
model.save("triage_model.h5")

# Scaler'ı kaydet (pickle formatı)
joblib.dump(scaler, "scaler.pkl")

print("Model ve scaler başarıyla kaydedildi.")

In [ ]:
import gradio as gr
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# === MODEL YÜKLE ===
model = load_model("/content/triage_model.h5")
scaler = joblib.load("/content/scaler.pkl")

renk_map = {0: "🔴 Kırmızı (Acil)", 1: "🟡 Sarı (Orta)", 2: "🟢 Yeşil (Düşük)"}

# === SEÇENEKLERİN SAYISAL KARŞILIKLARI ===
cinsiyet_map = {"Erkek": 1, "Kadın": 2}
gelis_map = {
    "Yürüyerek geldi": 1,
    "Ambulans ile geldi": 2,
    "Başka sağlık kurumundan sevk": 3
}
travma_map = {"Hayır (Travmasız)": 1, "Evet (Travmalı)": 2}
cikis_map = {
    "Acilde kaldı / Ayaktan taburcu": 1,
    "Yatış yaptı veya başka yere sevk edildi": 2
}

# === TAHMİN FONKSİYONU ===
def tahmin_et(
    grup, cinsiyet, yas, saatlik_hasta, gelis, travma, mental, agri_var,
    nrs, sbp, dbp, hr, rr, bt, spo2, ktas_rn, cikis, hata, kalis_dk, sure_dk, yanlis_mi
):
    veri = {
        "Group": 1 if grup == "Yetişkin" else 2,
        "Sex": cinsiyet_map[cinsiyet],
        "Age": yas,
        "Patients number per hour": saatlik_hasta,
        "Arrival mode": gelis_map[gelis],
        "Injury": travma_map[travma],
        "Mental": mental,
        "Pain": agri_var,
        "NRS_pain": nrs,
        "SBP": sbp,
        "DBP": dbp,
        "HR": hr,
        "RR": rr,
        "BT": bt,
        "Saturation": spo2,
        "KTAS_RN": ktas_rn,
        "Disposition": cikis_map[cikis],
        "Error_group": hata,
        "Length of stay_min": kalis_dk,
        "KTAS duration_min": sure_dk,
        "mistriage": yanlis_mi
    }

    df = pd.DataFrame([veri])
    scaled = scaler.transform(df)
    pred = model.predict(scaled)
    sinif = np.argmax(pred)
    return renk_map[sinif]

# === ARAYÜZ ===
app = gr.Interface(
    fn=tahmin_et,
    inputs=[
        gr.Dropdown(["Yetişkin", "Çocuk"], label="Hasta grubu"),
        gr.Dropdown(["Erkek", "Kadın"], label="Cinsiyet"),
        gr.Number(label="Yaş (örnek: 65)"),
        gr.Number(label="Saatlik hasta yoğunluğu (örnek: 5)"),
        gr.Dropdown(["Yürüyerek geldi", "Ambulans ile geldi", "Başka sağlık kurumundan sevk"], label="Hastanın geliş şekli"),
        gr.Dropdown(["Hayır (Travmasız)", "Evet (Travmalı)"], label="Hastada travma var mı?"),
        gr.Number(label="Zihinsel durum (1: Normal, 2: Bilinç bulanıklığı)", value=1),
        gr.Number(label="Ağrısı var mı? (1: Hayır, 2: Evet)", value=1),
        gr.Number(label="Ağrı şiddeti (0–10 arası)", value=3),
        gr.Number(label="Sistolik Kan Basıncı (örnek: 120)"),
        gr.Number(label="Diastolik Kan Basıncı (örnek: 80)"),
        gr.Number(label="Nabız (örnek: 75)"),
        gr.Number(label="Solunum Sayısı (örnek: 16)"),
        gr.Number(label="Vücut Isısı (°C, örnek: 36.8)"),
        gr.Number(label="Oksijen Saturasyonu (örnek: 98)"),
        gr.Number(label="Hemşire KTAS Puanı (1–5)", value=3),
        gr.Dropdown(["Acilde kaldı / Ayaktan taburcu", "Yatış yaptı veya başka yere sevk edildi"], label="Çıkış durumu"),
        gr.Number(label="Hata grubu (örnek: 2)"),
        gr.Number(label="Hastanede kalış süresi (dakika)", value=60),
        gr.Number(label="KTAS değerlendirme süresi (dakika)", value=5.0),
        gr.Number(label="Yanlış sınıflandırma mı? (1=Evet, 0=Hayır)", value=0)
    ],
    outputs=gr.Textbox(label="🩺 Tahmin Edilen Aciliyet Seviyesi"),
    title="🏥 Acil Triage Tahmin Uygulaması",
    description="Aşağıdaki hasta bilgilerini girerek sistemin tahmin ettiği aciliyet düzeyini (Kırmızı, Sarı, Yeşil) öğrenebilirsiniz."
)

app.launch()
